In [ ]:
import pandas as pd
data = pd.read_csv('data/examples/sample_full_traj_59-Scan-50%_edge_node_df.csv')  # Update 'your_data.csv' with your CSV file path
locations = data[['latitude', 'longitude']]